# Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
import re
import string
import gensim
from gensim.models import FastText
import random
from tabulate import tabulate
from tqdm import tqdm
import tensorflow as tf

In [ ]:
import nltk
import subprocess
try:
    nltk.data.find('wordnet.zip')
except:
    nltk.download('punkt', download_dir='/kaggle/working/')
    nltk.download('wordnet', download_dir='/kaggle/working/')
    nltk.download('stopwords', download_dir='/kaggle/working/')
    nltk.download('wordnet', download_dir='/kaggle/working/')
    nltk.download('averaged_perceptron_tagger', download_dir='/kaggle/working/')
    
    command = "unzip /kaggle/working/corpora/wordnet.zip -d /kaggle/working/corpora"
    subprocess.run(command.split())
    nltk.data.path.append('/kaggle/working/')
    
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Reading the file

In [ ]:
df = pd.read_csv('/kaggle/input/sentiment140/training.1600000.processed.noemoticon.csv',
                 encoding = 'latin',header=None)

df.head()

In [ ]:
df.shape

In [ ]:
df.columns = ['sentiment', 'id', 'date', 'query', 'user_id', 'text']
df.head()

In [ ]:
df = df.drop(['id', 'date', 'query', 'user_id'], axis=1)
df.head()

In [ ]:
# Define a dictionary to map sentiment values
sentiment_mapping = {0: "Negative", 4: "Positive"}

# Map the 'sentiment' column using the defined dictionary
df['sentiment'] = df['sentiment'].map(sentiment_mapping)

df.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Count the occurrences of each sentiment category
sentiment_counts = df['sentiment'].value_counts()

# Plotting the bar chart
plt.figure(figsize=(8, 6))
sns.barplot(x=sentiment_counts.index, y=sentiment_counts.values)

# Adding labels and title
plt.title('Distribution of Sentiments')
plt.xlabel('Sentiment')
plt.ylabel('Count')

# Show plot
plt.show()

# Taking a sample from the data

In [ ]:
# Generate a sample DataFrame with the same distribution of classes
sample_size = 500000

# Randomly sample 100000 rows from each class
sampled_df = df.groupby('sentiment', group_keys=False).apply(lambda x: x.sample(int(sample_size/2)))

# Reset the index of the sampled DataFrame
sampled_df.reset_index(drop=True, inplace=True)

# Count the occurrences of each sentiment category in the sampled DataFrame
sentiment_counts = sampled_df['sentiment'].value_counts()

# Plotting the bar chart
plt.figure(figsize=(8, 6))
sns.barplot(x=sentiment_counts.index, y=sentiment_counts.values)

# Adding labels and title
plt.title('Distribution of Sentiments in Sampled DataFrame')
plt.xlabel('Sentiment')
plt.ylabel('Count')

# Show plot
plt.show()

# Applying Preprocessing on the data

In [ ]:
def remove_usernames(text):
    # Remove usernames (mentions) from the text
    return re.sub(r'@\w+', '', text)

def tokenize_text(text):
    # Tokenize the text into words
    tokens = word_tokenize(text)
    return tokens

def normalize_tokens(tokens):
    # Normalize tokens to lowercase
    normalized_tokens = [token.lower() for token in tokens]
    return normalized_tokens

def clean_tokens(tokens):
    # Filter out non-alphanumeric tokens
    cleaned_tokens = [token for token in tokens if token.isalnum() and not token.isdigit()]
    return cleaned_tokens

def remove_stopwords(tokens):
    # Remove stop words from tokens
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token not in stop_words]
    return filtered_tokens

def lemmatize_tokens(tokens):
    # Lemmatize tokens based on part of speech
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = []
    for token in tokens:
        pos_tag = nltk.pos_tag([token])[0][1][0].upper()
        pos_map = {"N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV, "J": wordnet.ADJ}
        wordnet_pos = pos_map.get(pos_tag, wordnet.NOUN)
        lemma = lemmatizer.lemmatize(token, pos=wordnet_pos)
        lemmatized_tokens.append(lemma)
    return lemmatized_tokens

def filter_meaningful_tokens(tokens):
    # Filter tokens based on length and presence in WordNet
    meaningful_tokens = [token for token in tokens if len(token) >= 3]
    return meaningful_tokens

def preprocess_text(text):
    # Step 1: Remove usernames
    text = remove_usernames(text)
     # Step 2: Tokenize text
    tokens = tokenize_text(text)
    
    # Step 3: Normalize tokens
    normalized_tokens = normalize_tokens(tokens)
    
    # Step 4: Clean tokens
    cleaned_tokens = clean_tokens(normalized_tokens)
    
    # Step 5: Remove stop words
    filtered_tokens = remove_stopwords(cleaned_tokens)
    
    # Step 6: Lemmatize tokens
    lemmatized_tokens = lemmatize_tokens(filtered_tokens)
    
    # Step 7: Filter meaningful tokens
    meaningful_tokens = filter_meaningful_tokens(lemmatized_tokens)
    
    # Join tokens into preprocessed text
    preprocessed_text = ' '.join(meaningful_tokens)
    
    return preprocessed_text

In [ ]:
import pandas as pd
from tqdm import tqdm

# Initialize tqdm with pandas apply function
tqdm.pandas()

# Assuming sampled_df is your DataFrame and preprocess_text is your preprocessing function
sampled_df['cleaned_text'] = sampled_df['text'].progress_apply(preprocess_text)

# Display the DataFrame with cleaned text
print(sampled_df.head(10))

In [ ]:
# Define the file path and name for the CSV file
csv_file_path = '/kaggle/working/sample_dataframe.csv'

# Export the DataFrame to CSV
sampled_df.to_csv(csv_file_path, index=False)

print(f"DataFrame has been exported to {csv_file_path}.")

# Splitting the data

In [ ]:
from sklearn.model_selection import train_test_split

# Splitting the data into features (X) and target (y)
X = sampled_df['cleaned_text']
y = sampled_df['sentiment']

# Splitting the data into training (80%) and testing (20%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y , random_state=42)

# Convert sentiment labels to numerical format
y_train_numeric = (y_train == 'Positive').astype(int)
y_test_numeric = (y_test == 'Positive').astype(int)

# Displaying the shapes of the new DataFrames
print("Training data shape:", X_train.shape)
print("Testing data shape:", X_test.shape)

# Usign FastText to get the embeddings of the data

In [ ]:
# Convert 'cleaned_text' from training data (X_train) to list of sentences
train_sentences = [text.split() for text in X_train]

# Train FastText model
model = FastText(sentences=train_sentences, vector_size=30, window=2, min_count=1, sg=1, epochs=100)

# Preparing the data before feeding it into the Model

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
import pickle

# Tokenize and convert text data to sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

# Save the tokenizer after fitting
with open('tokenizer.pkl', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

print("Tokenizer saved to tokenizer.pkl")

In [ ]:
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Pad sequences to ensure uniform length
max_len = 30
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_len)
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_len)

In [ ]:
# Prepare embedding matrix from FastText model
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1
embedding_matrix = np.zeros((vocab_size, 30))  # vector_size=30 in FastText
for word, i in word_index.items():
    if word in model.wv:
        embedding_matrix[i] = model.wv[word]

# Model Preparing and Training

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.initializers import Constant

# Build the simplified LSTM model
model = Sequential()

# Embedding layer using the embedding_matrix
embedding_layer = Embedding(input_dim=vocab_size, 
                             output_dim=30, 
                             embeddings_initializer=Constant(embedding_matrix), 
                             input_length=max_len, 
                             trainable=True)  # Set trainable to True if you want to fine-tune the embeddings
model.add(embedding_layer)

# Single LSTM layer
model.add(LSTM(50, dropout=0.2))  # Reduced units and dropout

# Fully connected layer with fewer units
model.add(Dense(8, activation='relu'))  # Reduced units
# Output layer for binary classification
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

In [ ]:
# Train the model with numerical labels and store the training history
history = model.fit(X_train_padded, y_train_numeric, epochs=5, batch_size=64, validation_split=0.2)

In [ ]:
# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plot training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Model Evaluation

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

# Predict classes on test data
y_pred = (model.predict(X_test_padded) > 0.5).astype(int)

# Create confusion matrix
conf_matrix = confusion_matrix(y_test_numeric, y_pred)

# Plot confusion matrix as a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Negative', 'Positive'], 
            yticklabels=['Negative', 'Positive'])
plt.title('Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

# Display classification report
report = classification_report(y_test_numeric, y_pred)
print("Classification Report:")
print(report)

# Model Testing Example

In [ ]:
# Ensure X_test and y_test_numeric are numpy arrays
X_test_array = X_test.values if isinstance(X_test, pd.Series) else np.array(X_test)
y_test_array = y_test_numeric.values if isinstance(y_test_numeric, pd.Series) else np.array(y_test_numeric)

# Select 10 random indices
random_indices = random.sample(range(len(X_test_array)), 10)

# Prepare data for the table
table_data = []
for idx, rand_idx in enumerate(random_indices, start=1):
    sentence = X_test_array[rand_idx]
    true_label = "Positive" if y_test_array[rand_idx] == 1 else "Negative"
    predicted_label = "Positive" if y_pred[rand_idx] == 1 else "Negative"
    table_data.append([idx, sentence, true_label, predicted_label])

# Create headers for the table
headers = ["Index", "Sentence", "True Label", "Predicted Label"]

# Display the table with 'fancy_grid' format
print(tabulate(table_data, headers=headers, tablefmt="fancy_grid"))

# Saving the Model

In [ ]:
# Save the model in HDF5 format
model.save('sentiment_analysis_model.h5')